# TNT Cates (Living Polymers): Startup Shear

## Objectives
- Fit TNTCates model to startup shear stress data
- Analyze stress overshoot from reptation and scission dynamics
- Compare with single-mode Maxwell at effective time tau_d
- Show interplay between breaking and reptation
- Perform Bayesian inference with NUTS

## Setup

In [ ]:
import os
import sys
import time

# CI mode for automated testing
FAST_MODE = os.environ.get("FAST_MODE", "0") == "1"

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_pnas_startup,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_cates_param_names,
    compute_cates_tau_d,
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

## Theory: Cates Model for Startup Shear

The Cates model describes startup transients in living polymers with coupled dynamics:

**Governing equations:**
$$\frac{\partial \tau_{ij}}{\partial t} + \tau_{ij}/\tau_{\text{break}} = G_0 \dot{\gamma}_{ij} - \frac{1}{\tau_{\text{rep}}} \sum_k (\tau_{ik}\tau_{kj} - \delta_{ij}\text{tr}(\tau^2)/3)$$

**Key features:**
- Stress overshoot from reptation dynamics
- Breaking relaxes stress on time scale $\tau_{\text{break}}$
- Effective relaxation $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$
- In fast-breaking limit, similar to Maxwell but with scission effects

**Startup shear protocol:**
- Step shear rate $\dot{\gamma}$ applied at $t=0$
- Measure stress evolution $\sigma(t)$ to steady state

## Load Startup Shear Data

In [ ]:
gamma_dot = 1.0
time_data, stress = load_pnas_startup(gamma_dot=gamma_dot)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} to {time_data.max():.2e} s")
print(f"Applied shear rate: {gamma_dot:.2f} s^-1")
print(f"Stress range: {stress.min():.2f} to {stress.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(time_data, stress, 'o', label='Experimental data', markersize=5)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title(f'Startup Shear Data (γ̇ = {gamma_dot:.2f} s$^{{-1}}$)', fontsize=14)
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

if FAST_MODE:
    print("FAST_MODE: Using default parameters (NLSQ fit for startup is slow)")
    # Set reasonable default parameters for TNTCates startup
    model.parameters.set_value('G_0', 100.0)
    model.parameters.set_value('tau_rep', 0.1)
    model.parameters.set_value('tau_break', 1.0)
    model.parameters.set_value('eta_s', 0.01)
    fit_time = 0.0
else:
    start_time = time.time()
    model.fit(time_data, stress, test_mode='startup', gamma_dot=gamma_dot, method='scipy')
    fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
stress_pred_fit = model.predict(time_data, test_mode='startup', gamma_dot=gamma_dot)
quality = compute_fit_quality(stress, stress_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
time_pred = jnp.linspace(time_data.min(), time_data.max(), 300)
stress_pred = model.predict(time_pred, test_mode='startup', gamma_dot=gamma_dot)

# Compute residuals
residuals = stress - model.predict(time_data, test_mode='startup', gamma_dot=gamma_dot)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, stress, 'o', label='Data', markersize=5)
ax1.plot(time_pred, stress_pred, '-', linewidth=2, label='TNTCates fit')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Startup Shear Fit', fontsize=14)

ax2.plot(time_data, residuals, 'o', markersize=4)
ax2.axhline(0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Residuals (Pa)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_title('Residuals', fontsize=14)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Physical Analysis

In [ ]:
tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

overshoot_idx = jnp.argmax(stress_pred)
overshoot_time = time_pred[overshoot_idx]
overshoot_stress = stress_pred[overshoot_idx]
steady_stress = stress_pred[-1]

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")
print(f"\nStress Overshoot:")
print(f"  Peak time: {overshoot_time:.4f} s")
print(f"  Peak stress: {overshoot_stress:.2f} Pa")
print(f"  Steady stress: {steady_stress:.2f} Pa")
print(f"  Overshoot ratio: {overshoot_stress/steady_stress:.3f}")

if zeta < 0.1:
    print(f"\n  → Fast-breaking limit: Overshoot dominated by reptation")
else:
    print(f"\n  → Full Cates dynamics: Coupled reptation and scission")

## Compare with Single-Mode Maxwell

In [ ]:
from rheojax.models.tnt import TNTSingleMode

maxwell_model = TNTSingleMode()
# Set Maxwell parameters (TNTSingleMode uses G, tau_b, eta_s)
maxwell_model.parameters.set_value('G', nlsq_params['G_0'])
maxwell_model.parameters.set_value('tau_b', tau_d)
maxwell_model.parameters.set_value('eta_s', nlsq_params['eta_s'])

maxwell_pred = maxwell_model.predict(time_pred, test_mode='startup', gamma_dot=gamma_dot)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress, 'o', label='Data', markersize=5, zorder=3)
ax.plot(time_pred, stress_pred, '-', linewidth=2, label='TNTCates', zorder=2)
ax.plot(time_pred, maxwell_pred, '--', linewidth=2, label=f'Maxwell (τ={tau_d:.2e}s)', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Cates vs Maxwell Comparison', fontsize=14)
plt.close("all")
plt.close('all')

## Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        time_data if 'time_data' in dir() else t_data,
        stress if 'stress' in dir() else strain,
        test_mode='startup',
        gamma_dot=gamma_dot,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Trace Plot

In [ ]:
# Skip trace plot in CI mode (placeholder data not suitable for visualization)
if not FAST_MODE:
    idata = az.from_dict(posterior={k: v.reshape(NUM_CHAINS, NUM_SAMPLES) for k, v in posterior.items()})
    
    axes = az.plot_trace(idata, var_names=param_names, compact=False, figsize=(12, 8))
    plt.suptitle('MCMC Trace Plots', fontsize=14, y=1.0)
    plt.tight_layout()
    plt.close("all")
    plt.close('all')
else:
    print(f"FAST_MODE: Skipping trace plot (using NLSQ point estimates)")


## ArviZ Pair Plot

In [ ]:
# Skip pair plot in CI mode (placeholder data not suitable for visualization)
if not FAST_MODE:
    axes = az.plot_pair(
        idata,
        var_names=param_names,
        kind='kde',
        marginals=True,
        figsize=(10, 10)
    )
    plt.suptitle('Posterior Correlations', fontsize=14, y=0.995)
    plt.close("all")
    plt.close('all')
else:
    print(f"FAST_MODE: Skipping pair plot (using NLSQ point estimates)")


## ArviZ Forest Plot

In [ ]:
# Skip forest plot in CI mode (placeholder data not suitable for visualization)
if not FAST_MODE:
    axes = az.plot_forest(
        idata,
        var_names=param_names,
        combined=True,
        hdi_prob=0.95,
        figsize=(8, 4)
    )
    plt.suptitle('95% Credible Intervals', fontsize=14)
    plt.close("all")
    plt.close('all')
else:
    print(f"FAST_MODE: Skipping forest plot (using NLSQ point estimates)")


## Posterior Predictive Distribution

In [ ]:
if FAST_MODE:
    print("FAST_MODE: Skipping posterior predictive (200 ODE predictions would take >300s)")
    # Single NLSQ prediction for plot
    stress_pred_final = model.predict(time_pred, test_mode='startup', gamma_dot=gamma_dot)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(time_data, stress, 'o', label='Data', markersize=5)
    ax.plot(time_pred, stress_pred_final, '-', linewidth=2, label='NLSQ fit')
    ax.set_xlabel('Time (s)', fontsize=12)
    ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_title('NLSQ Fit (CI Mode)', fontsize=14)
    plt.close("all")
    plt.close('all')
else:
    n_posterior_samples = 200
    sample_indices = np.random.choice(NUM_SAMPLES * NUM_CHAINS, n_posterior_samples, replace=False)

    predictions = []
    for idx in sample_indices:
        # Set model parameters from posterior sample
        for j, name in enumerate(param_names):
            model.parameters.set_value(name, float(posterior[name].flatten()[idx]))
        # Predict with updated parameters
        pred_i = model.predict(time_pred, test_mode='startup', gamma_dot=gamma_dot)
        predictions.append(np.array(pred_i))

    predictions = np.array(predictions)
    pred_mean = np.mean(predictions, axis=0)
    pred_lower = np.percentile(predictions, 2.5, axis=0)
    pred_upper = np.percentile(predictions, 97.5, axis=0)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(time_data, stress, 'o', label='Data', markersize=5, zorder=3)
    ax.plot(time_pred, pred_mean, '-', linewidth=2, label='Posterior mean', zorder=2)
    ax.fill_between(time_pred, pred_lower, pred_upper, alpha=0.3, label='95% CI', zorder=1)
    ax.set_xlabel('Time (s)', fontsize=12)
    ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_title('Posterior Predictive Distribution', fontsize=14)
    plt.close("all")
    plt.close('all')

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(zeta_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(zeta_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(0.1, color='g', linestyle=':', label='Fast-breaking limit')
ax2.set_xlabel(r'$\zeta = \tau_{break}/\tau_{rep}$', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Fast-Breaking Parameter Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "startup", param_names)

## Key Takeaways

1. **Stress overshoot** in Cates model arises from coupled reptation and scission dynamics
2. **Effective relaxation time** $\tau_d$ governs overshoot timing, similar to Maxwell but with breaking effects
3. **Fast-breaking limit** gives Maxwell-like behavior with $\tau = \tau_d$
4. **Overshoot ratio** depends on $\zeta = \tau_{\text{break}}/\tau_{\text{rep}}$
5. **Bayesian inference** captures uncertainty in time scales from transient data

**Next steps:** Compare extracted time scales with SAOS data (NB11) to validate consistency across protocols.